In [1]:
import sys
import importlib
import numpy as np
from saxstats import saxstats
from Ligand import Ligand
from Geometry import *
import rdkit
from rdkit import Chem
import time
from array import array
from scipy.optimize import minimize
from spyrmsd import rmsd as srmsd

In [4]:
lig_gt = Ligand('1FIN_ligand.sdf')

This ligand does not seem to have hydrogens modeled into them
Consider doing so for best X-ray scattering signal accuracy


[10:18:22] Molecule does not have explicit Hs. Consider calling AddHs()


In [3]:
sp = {'a': np.zeros(lig_gt.num_angle), 't': np.mean(lig_gt.get_coordinates(), 0)}
sp['a'][0] = 30

In [4]:
lig_gt.agroup

[[array([0, 7, 4]),
  array([0, 1, 2, 3]),
  array([ 4,  5,  6,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
         22, 23, 24, 25, 26, 27, 28, 29, 30])],
 [array([ 4, 11,  8]),
  array([0, 1, 2, 3, 4, 5, 6, 7]),
  array([ 8,  9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
         26, 27, 28, 29, 30])],
 [array([ 7,  4, 11]),
  array([0, 1, 2, 3, 7]),
  array([ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
         25, 26, 27, 28, 29, 30])],
 [array([ 8, 12, 13]),
  array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]),
  array([13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
         30])],
 [array([11,  8, 12]),
  array([ 0,  1,  2,  3,  4,  5,  6,  7, 11]),
  array([12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
         29, 30])],
 [array([12, 13, 14]),
  array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
  array([14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30

In [5]:
lig_gt.set_coordinates(lig_gt.transform(0, sp))

In [7]:
lig_gt.save('test_angle.pdb')

In [2]:
lig_gt = Ligand('1FIN_ligand.sdf')

This ligand does not seem to have hydrogens modeled into them
Consider doing so for best X-ray scattering signal accuracy


[10:19:21] Molecule does not have explicit Hs. Consider calling AddHs()


In [3]:
def x_to_dict(x, lig):
    return {'t': x[:3], 'r': x[3:6], 'a': x[6:6+lig.num_angle], 'd': x[6+lig.num_angle:]}
def myminfunc(x, lig, lig_coord_gt):
    x_dict = x_to_dict(x, lig)
    return pureRMSD(lig.transform(0, x_dict), lig_coord_gt)
def mysymmminfunc(x, lig, lig_coord_gt, isomorphism=None, return_isomorphism=False):
    x_dict = x_to_dict(x, lig)
    if return_isomorphism:
        return symmRMSD(lig.transform(0, x_dict), lig_coord_gt, lig.A, lig.G, isomorphism)
    else:
        return symmRMSD(lig.transform(0, x_dict), lig_coord_gt, lig.A, lig.G, isomorphism)[0]

In [4]:
sp_np = {'t': np.random.random(3) * 180,
         'r': np.random.random(3) * 180,
         'a': np.random.random(lig_gt.num_angle) * 10,
         'd': np.random.random(lig_gt.num_torsion) * 180}

In [5]:
# lig_gt.set_coordinates(lig_gt.transform(0, sp))
lig_coord_gt = lig_gt.get_coordinates()

In [6]:
# res = minimize(myminfunc, np.zeros(6+lig.num_angle+lig.num_torsion), 
#                args=(lig, lig_coord_gt), 
#                method='L-BFGS-B', 
#                options={'iprint':10, 'maxfun': 100000, 'gtol': 3e-3})
#                #bounds=[(None, None), (None, None), (None, None), (0, 180), (0, 360), 
#                        #*([(None, None)] * lig.num_torsion)], options={'iprint': 10})

# print(res)

In [7]:
iso=None

In [8]:
lig = Ligand('1FIN_ligand.sdf')
lig.generate_conformers(1)
# lig.get_coordinates()
# lig.set_coordinates(lig.transform(0, sp_np))

This ligand does not seem to have hydrogens modeled into them
Consider doing so for best X-ray scattering signal accuracy
Generated 1 conformers


[10:19:25] Molecule does not have explicit Hs. Consider calling AddHs()
[10:19:25] Molecule does not have explicit Hs. Consider calling AddHs()


In [9]:
rmsd, iso = mysymmminfunc(np.zeros(6+lig.num_angle+lig.num_torsion),
                          lig, lig_coord_gt, isomorphism=None, return_isomorphism=True)
print(rmsd)

236.63701391513234


In [10]:
# Run with bounds on angles
res = minimize(mysymmminfunc, np.zeros(6+lig.num_angle+lig.num_torsion), 
               args=(lig, lig_coord_gt, iso), method='L-BFGS-B', 
               options={'iprint':50, 'maxfun': 100000, 'gtol': 1e-4},
               bounds=[(None, None)] * 6 + [(-0.01, 0.01)] * lig.num_angle + [(None, None)] * lig.num_torsion)
#                bounds=[(None, None), (None, None), (None, None), (0, 180), (0, 360), 
                       #*([(None, None)] * lig.num_torsion)], options={'iprint': 10})

print(res)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           20     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.36637D+02    |proj g|=  8.68442D-01

At iterate   50    f=  2.36658D+00    |proj g|=  1.97582D-02

At iterate  100    f=  1.61181D+00    |proj g|=  1.58430D-02

At iterate  150    f=  1.39755D+00    |proj g|=  3.01579D-03

At iterate  200    f=  1.34032D+00    |proj g|=  6.62901D-03

At iterate  250    f=  1.18206D+00    |proj g|=  2.10218D-03

At iterate  300    f=  1.06964D+00    |proj g|=  6.10465D-03

At iterate  350    f=  1.06137D+00    |proj g|=  1.82683D-03

At iterate  400    f=  1.05864D+00    |proj g|=  2.40559D-03

At iterate  450    f=  1.05313D+00    |proj g|=  5.25158D-03

At iterate  500    f=  1.04385D+00    |proj g|=  1.99229D-03

At iterate  550    f=  1.02772D+00    |proj g|=  1.04134D-02

At iterate  600    f=  7.65950D-01    |proj g|=  5.79120D-02

At iterate  650    f=  6.7

In [ ]:
res = minimize(mysymmminfunc, np.zeros(6+lig.num_angle+lig.num_torsion), 
               args=(lig, lig_coord_gt, iso), method='L-BFGS-B', 
               options={'iprint':50, 'maxfun': 100000, 'gtol': 1e-4})
               #bounds=[(None, None), (None, None), (None, None), (0, 180), (0, 360), 
                       #*([(None, None)] * lig.num_torsion)], options={'iprint': 10})

print(res)

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           20     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.36637D+02    |proj g|=  8.68442D-01


 This problem is unconstrained.


In [11]:
lig.set_coordinates(lig.transform(0, x_to_dict(res.x, lig)))
lig.save('test_min.pdb')

In [45]:
from spyrmsd import rmsd

In [47]:
lig.

array(['P', 'O', 'O', 'O', 'P', 'O', 'O', 'O', 'P', 'O', 'O', 'O', 'O',
       'C', 'C', 'O', 'C', 'O', 'C', 'O', 'C', 'N', 'C', 'N', 'C', 'C',
       'N', 'N', 'C', 'N', 'C'], dtype='<U1')

In [10]:
# print("Ground truth")
# print(lig_coord_gt)

# print("Minimized result")
# print(lig.transform(0, res.x))

print("Initial Kabsch RMSD:", end=' ')
print(Kabsch_RMSD(lig_coord_gt, lig.get_coordinates())[0])

print("Final RMSD:", end=' ')
print(pureRMSD(lig_coord_gt, lig.transform(0, res.x)))

print("Final Kabsch RMSD: ", end=' ')
print(Kabsch_RMSD(lig_coord_gt, lig.transform(0, res.x))[0])

Initial Kabsch RMSD: 2.1915280722526695
Final RMSD: 0.5632987965427904
Final Kabsch RMSD:  0.5632987962481528
